In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import trim, col # It's not good to import this line becuase we import everything in the first line already.

# Read From Bronze Table

In [0]:
df = spark.table("workspace.bronze.crm_cust_info")

# Data Transformations

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))


df.display()

# Write Into Silver Table